In [20]:
import sys; sys.path.append('methods/')
import os
import pandas
from regridding import *

In [10]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# define coordinate reference systems
osgb_crs = ccrs.TransverseMercator(approx = False, central_longitude = -2, central_latitude = 49, scale_factor = 0.9996012717, false_easting = 400000, false_northing = -100000,
                                   globe = ccrs.Globe(datum = 'OSGB36', ellipse = 'airy'))
latlon_crs = ccrs.RotatedPole(central_rotated_longitude = 180)

In [11]:
# #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# def nearest_px(x,y,da):
   
#     # get squared distance from (x,y) to each point
#     dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
#     # exclude any cells where the gridded data is NaN
#     dist2 = dist2.where(~np.isnan(da))
   
#     # also limit distance to closest two squares (in case there really is no data nearby)
#     dist2 = dist2.where(dist2 <= 0.125)
   
#     # find value in cell containing minimum distance
#     # if multiple equidistant cells, will average over them
#     val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
#     return val

In [12]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2019 constituencies
# sf_old = geopandas.read_file("Westminster_Parliamentary_Constituencies_December_2019_Boundaries_UK_BUC")
# sf_old

In [13]:
filesnames=["CORDEX-nearterm_rcp45_pr_DJF.nc","CORDEX-nearterm_rcp45_pr_JJA.nc","CORDEX-nearterm_rcp45_rx5day.nc","CORDEX-nearterm_rcp45_temp_ANN.nc","CORDEX-nearterm_rcp45_txx.nc","E-OBS_change_pr_DJF.nc","E-OBS_change_pr_JJA.nc","E-OBS_change_rx5day.nc","E-OBS_change_temp_ANN.nc","E-OBS_change_txx.nc"]
varnames_in=["pr_relanom","pr_relanom","rx5day_relanom","t_anom","txx_anom","pr_relanom","pr_relanom","rx5day_relanom","t_anom","txx_anom"]
varnames_out=["prdjf_proj","prjja_proj","rx5day_proj","t_proj","txx_proj","prdjf_obs","prjja_obs","rx5day_obs","t_obs","txx_obs"]

In [14]:
# for i in range(filesnames.__len__()):
#     fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
#     ds = xr.open_dataset(fnm)
#     #ds = add_grid(ds, "projection_x_coordinate", "projection_y_coordinate", osgb_crs, latlon_crs)
#     da=ds[varnames_in[i]]
#     #print(da)
#     rm = regionmask.mask_3D_geopandas(sf_old.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False, numbers = "objectid")
#     #print(rm)
#     # apply the regionmask to the data and average over the x & y dimensions
#     region_da = da.where(rm).mean(["lat", "lon"])
#     #print(region_da)
#     # find nearest neighbour (this takes the most time)
#     region_nn = xr.concat([nearest_px(sf_old.lat[i],sf.long[i],da).expand_dims(region = [sf_old.objectid[i]]).assign_coords(constituency = ("region", [sf_old.pcon19cd[i]])) for i in range(len(sf_old))], "region")
#     print(region_nn)
#     # combine regionmask with nearest neighbour where regionmask didn't pick anything up
#     region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
#     region_all=region_all.assign_coords(name=('region',sf_old.pcon19nm))
#     # output the values into a csv file
#     region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'_oldconstituencies.csv')
#     print("outputting ",varnames_out[i]+'_oldconstituencies.csv')




In [15]:
# Combine into a big dataframe
# import pandas
# all_files = os.listdir()    
# csv_files = list(filter(lambda f: f.endswith('_oldconstituencies.csv'), all_files))
# for i in range(csv_files.__len__()):
#     this_csv=pandas.read_csv(csv_files[i])
#     if i == 0:
#         csv_out=this_csv.iloc[:,[2,3,4]]
#     else:
#         segments=csv_files[i].split("_")
#         var='_'.join(segments[:2])
#         csv_out[var]=this_csv.iloc[:,4]

# csv_out.to_csv('../by_mp/rcp45_climate_oldconstituencies.csv',index=False)




In [16]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def nearest_px(x,y,da):
   
    print('x:',x,' y:',y)
    # get squared distance from (x,y) to each point
    dist2 = (da.lat - x)**2 + (da.lon - y)**2
   
    # exclude any cells where the gridded data is NaN
    dist2 = dist2.where(~np.isnan(da))
   
    # also limit distance to closest two squares (in case there really is no data nearby)
    dist2 = dist2.where(dist2 <= 0.125)
   
    # find value in cell containing minimum distance
    # if multiple equidistant cells, will average over them
    val = da.where(dist2 == dist2.min()).mean(["lat", "lon"])
   
    return val

In [17]:
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# load the shapefile for the 2024 constituencies
sf = geopandas.read_file("PCON_MAY_2024_UK_BFE")
sf

,PCON24CD,PCON24NM,PCON24NMW,BNG_E,BNG_N,LAT,LONG,GlobalID,geometry
0,E14001063,Aldershot,None,484716,155270,51.2903,-0.78648,b8b6c57e-6fe4-432c-9130-f4d6e4598d19,"POLYGON Z ((483364.601 160961.805 0.000, 48337..."
1,E14001064,Aldridge-Brownhills,None,404720,301030,52.6071,-1.93173,d3c0545a-9c68-4095-9b85-330c7141dbf2,"POLYGON Z ((406519.098 305054.298 0.000, 40648..."
2,E14001065,Altrincham and Sale West,None,374132,389051,53.3977,-2.39049,a7e18d2c-8dd0-4f1f-b158-48799d7f6259,"POLYGON Z ((377443.302 393344.296 0.000, 37745..."
3,E14001066,Amber Valley,None,440478,349674,53.0428,-1.39771,e38d2edf-19ce-4090-ab8e-c78449d3983f,"POLYGON Z ((436223.299 356984.804 0.000, 43624..."
4,E14001067,Arundel and South Downs,None,497309,118530,50.9580,-0.61585,4a4d2a3a-a7a5-4a91-a90f-0692b459c37c,"POLYGON Z ((505688.454 133874.110 0.000, 50569..."
...,...,...,...,...,...,...,...,...,...
645,W07000108,Swansea West,Gorllewin Abertawe,264670,195124,51.6386,-3.95702,ab9c216c-374a-4dda-aabb-b5d4095808c5,"POLYGON Z ((266627.504 191650.700 0.000, 26662..."
646,W07000109,Torfaen,Torfaen,327459,200480,51.6984,-3.05102,fe1b6677-bcf4-49d8-a516-85b42b10af00,"POLYGON Z ((333723.000 192653.903 0.000, 33372..."
647,W07000110,Vale of Glamorgan,Bro Morgannwg,301298,173080,51.4481,-3.42174,43710abb-9c64-41bd-9f14-6e8120252dc2,"POLYGON Z ((302260.803 179531.999 0.000, 30226..."
648,W07000111,Wrexham,Wrecsam,337298,348629,53.0313,-2.93642,95c39b06-f367-4432-9aaa-de2a48654814,"POLYGON Z ((323631.902 349914.197 0.000, 32363..."


In [18]:
for i in range(filesnames.__len__()):
    fnm = "../gridded_data/Copernicus_Atlas_data/"+filesnames[i]
    ds = xr.open_dataset(fnm)
    print(ds)
    da=ds[varnames_in[i]]
    rm = regionmask.mask_3D_geopandas(sf.to_crs(latlon_crs.proj4_init), ds.lon, ds.lat, drop = False)
    # apply the regionmask to the data and average over the x & y dimensions
    print(rm)
    region_da = da.where(rm).mean(["lat", "lon"])
    print(region_da)
    # find nearest neighbour (this takes the most time)
    region_nn = xr.concat([nearest_px(sf.LAT[i],sf.LONG[i],da).expand_dims(region = [region_da.region[i]]).assign_coords(constituency = ("region", [sf.PCON24CD[i]])) for i in range(len(sf))], "region")
    print(region_nn)
    # combine regionmask with nearest neighbour where regionmask didn't pick anything up
    region_all = xr.concat([region_da, region_nn.where(np.isnan(region_da))], "match").sum("match")
    region_all=region_all.assign_coords(constituency=('region',sf.PCON24CD))
    region_all=region_all.assign_coords(name=('region',sf.PCON24NM))
    # output the values into a csv file
    region_all.to_dataframe(name=varnames_out[i]).reset_index().to_csv(varnames_out[i]+'_newconstituencies.csv')
    print("outputting ",varnames_out[i]+'_newconstituencies.csv')

<xarray.Dataset>
Dimensions:     (lat: 92, lon: 88)
Coordinates:
  * lat         (lat) float64 49.56 49.69 49.81 49.94 ... 60.69 60.81 60.94
  * lon         (lon) float64 -8.938 -8.812 -8.688 -8.562 ... 1.688 1.812 1.938
Data variables:
    crs         |S1 ...
    pr_relanom  (lat, lon) float32 ...
Attributes: (12/29)
    Conventions:                CF-1.9 ACDD-1.3
    date_created:               2023-12-05 10:54:34.735166+01:00
    experiment_id:              rcp45
    frequency:                  mon
    geospatial_lat_max:         72.75
    geospatial_lat_min:         21.75
    ...                         ...
    title:                      Copernicus Interactive Climate Atlas: gridded...
    tracking_id:                04bad71f-e793-4456-8472-5f24e1f81b7a
    variable_id:                pr
    GDAL:                       GDAL 3.7.1, released 2023/07/06
    history:                    Thu Jun 20 21:25:14 2024: ncks -O -d lat,49.5...
    NCO:                        netCDF Operators ve

In [21]:
all_files = os.listdir()    
csv_files = list(filter(lambda f: f.endswith('_newconstituencies.csv'), all_files))
for i in range(csv_files.__len__()):
    this_csv=pandas.read_csv(csv_files[i])
    if i == 0:
        csv_out=this_csv.iloc[:,[2,3,4]]
    else:
        segments=csv_files[i].split("_")
        var='_'.join(segments[:2])
        csv_out[var]=this_csv.iloc[:,4]

csv_out.to_csv('../by_mp/rcp45_climate_newconstituencies.csv',index=False)